In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [8]:
# URL yang akan di scraping
URL = "https://liquipedia.net/mobilelegends/Portal:Heroes"

# Header untuk menghindari blocking 403
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Linux; Cashtree) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.114 Mobile Safari/537.36"
}

In [9]:
response = requests.get(URL, headers=HEADERS, timeout=15)

## Get Heroes List

In [12]:
if response.status_code != 200:
    print(f"Error: HTTP {response.status_code}")

soup = BeautifulSoup(response.content, "html.parser")

In [17]:
hero_table = soup.find("div", {"class": "sapphire-theme-dark-bg"})

In [21]:
heroes_data = hero_table.find_next_sibling()

In [39]:
heroes = []

In [28]:
base_img_url = "https://liquipedia.net"

In [40]:
if heroes_data:
    rows = heroes_data.find_all("div")
    for r in rows:
        hero_name_span = r.find("span")
        hero_name = hero_name_span.text.strip() if hero_name_span else "Unknown"
        
        hero_img_a = r.find("a")
        img = hero_img_a.find("img")
        hero_img = f"{base_img_url}{img["src"]}"

        hero_detail = f"{base_img_url}{hero_img_a["href"]}"
        

        hero = {"name": hero_name, "img": hero_img, "url": hero_detail}
        heroes.append(hero)
        

In [41]:
len(heroes)

131

## Get Hero Detail

In [280]:
hero_details = []

In [279]:
def get_detail_hero(name, url):
    print(f"get detail hero {name}...")
    hero_details = {"name": name}
    response = requests.get(url, headers=HEADERS, timeout=15)
    
    if response.status_code != 200:
        print(f"Error: HTTP {response.status_code}")
        return {}

    soup = BeautifulSoup(response.content, "html.parser")
    hero_detail = soup.find("div", {"id": "main-content"})
    content = hero_detail.find("div")
    main_content = content.find("div", {"id": "mw-content-text"})

    #overview
    overview = {"title": "Unknown", "description": "Unknown"}
    overview_body = main_content.find("div", {"class": "mw-content-ltr"})
    overview_check = overview_body.find_all("p")
    if len(overview_check) > 1:
        for o in overview_check:
            if o.find("button"):
                continue
                
            if o.find("i"):
              overview["title"] = main_content.find("p").text.strip()
            else: 
                if overview["title"] == "Unknown":
                    overview["title"] = o.find("b").text.strip() if o.find("b") else "Unknown"
                overview["description"] = o.get_text(separator=" ", strip=True).replace(overview["title"], "").strip() if o else "Unknown"
    else:
        overview_content = overview_body.find("p")
        print(overview_content)
        if overview_content is not None and overview_content.find("b") is not None:
            overview = {"title": overview_content.find("b").text.strip() if overview_content else "Unknown", "description": overview_content.get_text(separator=" ", strip=True).replace(overview_content.find("b").text.strip(), "").strip() if overview_content else "Unknown"}
    hero_details["overview"] = overview
    
    #abilities
    abilities = []
    abilities_elements = hero_detail.find_all("h3")
    ability_skip = ["Videos"]
    for a in abilities_elements:
        abiliti_name = a.find("span")
        if abiliti_name["id"] not in ability_skip:
            skill = a.find_next_sibling()
            ability_type = skill.find("div", {"class": "row"})
            skill_types = []
            if ability_type is None:
                skill_types = skill.find_all("div", {"class": "col-xs-2"})
            else:
                skill_types = ability_type.find_all("div")
            types = []
            for i, s in enumerate(skill_types):
                ab = {"key": s.find("b").text.strip()}
                details = s.find_all("b")
                if len(details) == 1:
                    ab["value"] = skill_types[i].text.strip()
                else:
                    c = ""
                    for d in details:
                        if d.find("span"):
                            c += f"{d.text} "
                    ab["value"] = c
                types.append(ab)
            alias = skill.find("div", {"class": "wiki-backgroundcolor-light"})
            hero_skill = {"skill": abiliti_name["id"], "key": skill.get("id"), "name": alias.text.strip() if alias else "Unknown", "ability_type": types}
            abilities.append(hero_skill)
    hero_details["abilities"] = abilities
    print(f"success get detail hero {name}...")
    return hero_details

In [ ]:
aamon = get_detail_hero(heroes[0]["name"], heroes[0]["url"])
print(aamon)

In [276]:
cek_hero = get_detail_hero(kalea["name"], kalea["url"])
print(cek_hero)

get detail hero Kalea...
None
success get detail hero Kalea...
{'name': 'Kalea', 'overview': {'title': 'Unknown', 'description': 'Unknown'}, 'abilities': []}


In [ ]:
for hero in heroes:
    detail = get_detail_hero(hero["name"], hero["url"])
    hero_details.append(detail)

In [282]:
len(hero_details)

131

In [283]:
df = pd.DataFrame(hero_details)

In [284]:
df

,name,overview,abilities
0,Aamon,{'title': 'The aloof Duke of Castle Aberleen.'...,"[{'skill': 'Passive', 'key': 'Invisible_Armor'..."
1,Akai,{'title': 'The almighty panda hero of Stream V...,"[{'skill': 'Passive', 'key': 'Tai_Chi', 'name'..."
2,Aldous,"{'title': 'Guardian of the Minoan Maze.', 'des...","[{'skill': 'Passive', 'key': 'Contract:_Transf..."
3,Alice,{'title': 'Cunning and sinister Blood Demon Qu...,"[{'skill': 'Passive', 'key': 'Blood_Ancestry',..."
4,Alpha,{'title': 'A pure cyborg made by Laboratory 17...,"[{'skill': 'Passive', 'key': 'Beta,_Advance!',..."
...,...,...,...
126,Yve,"{'title': 'Unknown', 'description': 'Yve, Astr...","[{'skill': 'Passive', 'key': 'Galactic_Power',..."
127,Zetian,"{'title': 'Unknown', 'description': ''}","[{'skill': 'Passive', 'key': 'Celestial_Armame..."
128,Zhask,{'title': 'King of the Swarm who disrupts the ...,"[{'skill': 'Passive', 'key': 'Decimation', 'na..."
129,Zhuxin,{'title': 'A mysterious young woman who guides...,"[{'skill': 'Passive', 'key': 'Crimson_Butterfl..."


In [285]:
heroes_df = pd.DataFrame(heroes)

In [286]:
heroes_df

,name,img,url
0,Aamon,https://liquipedia.net/commons/images/thumb/1/...,https://liquipedia.net/mobilelegends/Aamon
1,Akai,https://liquipedia.net/commons/images/thumb/a/...,https://liquipedia.net/mobilelegends/Akai
2,Aldous,https://liquipedia.net/commons/images/thumb/a/...,https://liquipedia.net/mobilelegends/Aldous
3,Alice,https://liquipedia.net/commons/images/thumb/5/...,https://liquipedia.net/mobilelegends/Alice
4,Alpha,https://liquipedia.net/commons/images/thumb/c/...,https://liquipedia.net/mobilelegends/Alpha
...,...,...,...
126,Yve,https://liquipedia.net/commons/images/thumb/f/...,https://liquipedia.net/mobilelegends/Yve
127,Zetian,https://liquipedia.net/commons/images/thumb/c/...,https://liquipedia.net/mobilelegends/Zetian
128,Zhask,https://liquipedia.net/commons/images/thumb/c/...,https://liquipedia.net/mobilelegends/Zhask
129,Zhuxin,https://liquipedia.net/commons/images/thumb/2/...,https://liquipedia.net/mobilelegends/Zhuxin
